In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [48]:
# input
idx_file="gwas-current-gz-wc.20200629-131527.tsv"
check402_file='20200703-gwas-402.1.check.tsv'

# output
check402_out_file='20200703-gwas-402.2.still-missing-402.tsv'



In [17]:
check402_df <- fread(check402_file)

In [27]:
check402_df %>%
rename('GBE_ID'='#GBE_ID') %>%
arrange(GBE_ID, population, f402) %>%
group_by(GBE_ID, population) %>%
summarise(f402=first(f402)) %>%
ungroup() -> check402_uniq_df


In [5]:
idx_df <- fread(idx_file) %>%
rename('symlink'='#symlink')


In [41]:
idx_df %>%
filter(wc_l == 1080567) %>%
separate(symlink, c(rep(NA, 9), 'population', 'basename'), sep='/', remove=F, fill='right', extra='drop') %>%
separate(basename, c(NA, 'GBE_ID', NA), sep='\\.', remove=T, fill='right', extra='drop') %>%
left_join(
    check402_uniq_df, by=c('GBE_ID', 'population')
) -> check_df


In [46]:
check_df %>% 
filter(is.na(f402)) %>%
dim()

[1] 345   6

In [49]:
check_df %>% 
filter(is.na(f402)) %>%
select(GBE_ID, population) %>%
rename('#GBE_ID' = 'GBE_ID') %>%
fwrite(check402_out_file, sep='\t', na = "NA", quote=F)
